In [13]:
import numpy as np
import argparse
from datasets.config import ModelConfig
from modules.basic_nn import BasicRS
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import FNNData
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm

In [14]:
set_seed(7)
dpath = 'data'

In [15]:
trainp = os.path.join(dpath, 'train.npy')
devp = os.path.join(dpath, 'valid.npy')

In [16]:
trainset = FNNData(np.load(trainp))
validset = FNNData(np.load(devp))

In [17]:
train_data_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
valid_data_loader = DataLoader(validset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [18]:
model = BasicRS(ModelConfig())

In [19]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [20]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
        data = data.type(torch.LongTensor)
        pred = model(data[:, 2:])
        loss = F.cross_entropy(pred, data[:, 1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))

EP-0 train loss: 1.3790627717971802:  10%|██████████▎                                                                                           | 425/4185 [01:00<08:56,  7.00it/s]Traceback (most recent call last):
  File "/home/azon/anaconda3/envs/coen344/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/home/azon/anaconda3/envs/coen344/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/azon/anaconda3/envs/coen344/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/home/azon/anaconda3/envs/coen344/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe

Traceback (most recent call last):
  File "/home/azon/anaconda3/envs/coen344/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/home/azon/anaco

KeyboardInterrupt: 